<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/project_TDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> TDA course project </h1>

Name: MRA

Pair: If you did this project as pair work, name the other student here, leave empty otherwise. If you work in pair, <b>both</b> hand out the same project report in Moodle.


<h1> Step 1: Load the data with LLM judgements </h1>

In [1]:
#work here
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00


In [2]:
import os
import gzip
import json
import pandas as pd
from datasets import Dataset

# Step 1: Download the file
url = "http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz"
filename = "tda25-responses.jsonl.gz"

os.system(f"wget {url} -O {filename}")

# Step 2: Extract the JSONL file
extracted_filename = "tda25-responses.jsonl"

with gzip.open(filename, 'rt', encoding='utf-8') as f_in:
    data = [json.loads(line) for line in f_in]

# Step 3: Convert to Pandas DataFrame
df = pd.DataFrame(data)

# Step 4: Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Verify the dataset
print(hf_dataset)


Dataset({
    features: ['document', 'response'],
    num_rows: 2515
})


In [3]:
print(hf_dataset[0])  # Print the first sample


{'document': 'Peeling an onion seems like an trivial task, but if you’ve never peeled an onion before, it can be quite intimidating. Don’t worry – it is pretty easy to peel an onion.\nYou can now learn how to peel an onion by following these illustrated step-by-step instructions.\nStep #1: Put the whole onion on the cutting board\nStep 2: Cut off one end of the onion with a knife, as shown on the picture below:\nHere’s a picture of the onion with that end already cut off. The end of the onion is laying on the right side of the onion on the cutting board.\nStep 3: Cut off another end of the onion with a knife, as show on the picture below:\nAfter both ends of the onion have been cut off, the onion is ready to be peeled. Here’s the picture of the onion without its ends:\nStep 4: Start peeling! Make a cut under the peel, and pull on the peel so it separates from the onion. Look at the picture: knife under the peel, thumb on top of the peel. Grab the peel and pull.\nStep 5: Keep peeling in

In [4]:
from transformers import AutoTokenizer

# Load the pre-trained GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the 'document' column
def tokenize_function(examples):
    return tokenizer(examples['document'], truncation=True, padding="max_length", max_length=512)

# Apply the tokenizer to the dataset
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)

# Check the tokenized dataset
print(tokenized_datasets)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'response', 'input_ids', 'attention_mask'],
    num_rows: 2515
})


In [5]:
# Prepare the dataset to be used in the fine-tuning
def format_for_finetuning(examples):
    return {
        'input_ids': examples['input_ids'],
        'attention_mask': examples['attention_mask'],
        'labels': examples['input_ids']  # We'll use the same input for labels for a causal language modeling task
    }

# Apply the formatting function
final_dataset = tokenized_datasets.map(format_for_finetuning, batched=True)

# Check the formatted dataset
print(final_dataset)


Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

Dataset({
    features: ['document', 'response', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2515
})


<h1> Step 2: Classifier training and evaluation </h1>


*   Which target did you choose?
*   Label distribution and majority baseline
*   Classifier performance
*   Manual inspection of the classifier output, what kinds of mistakes it makes?
*   What is the composition of the data we gave you? What does it mean for your results?
*   Concusions




In [ ]:
#work here

In [ ]:
from transformers import Trainer, TrainingArguments, GPT2LMHeadModel

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # Disable WandB
)


# Initialize Trainer
trainer = Trainer(
    model=model,                       # The pre-trained model
    args=training_args,                # Training arguments
    train_dataset=final_dataset,       # The training dataset
    eval_dataset=final_dataset         # The evaluation dataset
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,2.836400,2.350806
2,2.444400,2.241227
3,2.280300,2.203393


TrainOutput(global_step=3774, training_loss=2.479557213482556, metrics={'train_runtime': 1660.0005, 'train_samples_per_second': 4.545, 'train_steps_per_second': 2.273, 'total_flos': 1971448381440000.0, 'train_loss': 2.479557213482556, 'epoch': 3.0})

In [ ]:
# Evaluate the model
trainer.evaluate()


{'eval_loss': 2.203392505645752,
 'eval_runtime': 112.8824,
 'eval_samples_per_second': 22.28,
 'eval_steps_per_second': 11.144,
 'epoch': 3.0}

<h1> Bonus step </h1>

(leave empty if you do not do this)

*   Prompt design
*   Build (prompt,response pairs)
*   Turn into HF Dataset and save



In [ ]:
#work here

<h1> Summary and Conclusions </h1>

* Brief TL;DR -style summary and main conclusions of your project.

In [6]:
pip -q install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.9 MB/s eta 0:00:00


In [7]:
import openai
openai.api_key = "sk"

In [12]:
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

In [11]:
models = openai.Model.list()
for model in models['data']:
    print(model['id'])

gpt-4o-mini
text-embedding-3-small


In [14]:
#Write code to access the API, and retrieve results. Debug this with one short request until you know it works!
#import openai
import os
#os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxx"
# Make sure the API key is set correctly in the environment or use it directly like this:
#openai.api_key = os.getenv("OPENAI_API_KEY")  # This will fetch the key from the environment variable
# Alternatively, you can set it directly:
#openai.api_key = "sk-xxxxxxxxxxxx"

# Make the request to the model
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  # Use the correct model name
    messages=[
        {"role": "user", "content": "Just write : I have access to gpt-4o-mini, so you could luckily continue."}
    ]
)

print(response['choices'][0]['message']['content'])  # This will print the model's response

I have access to gpt-4o-mini, so you could luckily continue.


In [17]:
!pip -q install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [21]:
# Display the first few rows to check the structure
print(df.head())

# Display the available columns
print(df.columns)


                                            document  \
0  Peeling an onion seems like an trivial task, b...   
1  Cowboy’s WR Terrance Williams Arrested\nDallas...   
2  Idea Crib’s Well-thy Pinoy Profiles blog serie...   
3  Will I ever stop being nervous every time I pu...   
4  In today’s NHL rumors rundown, Toronto Maple L...   

                                            response  
0  Step-by-step: Yes  \nTraining: No  \n\nThe art...  
1  Step-by-step: No  \nTraining: No  \n\nThe arti...  
2  Step-by-step: No  \nTraining: No  \n\nThe arti...  
3  Step-by-step: No  \nTraining: No  \n\nThe arti...  
4  Step-by-step: No  \nTraining: No  \n\nThe arti...  
Index(['document', 'response'], dtype='object')


In [18]:
import evaluate
metric = evaluate.load("accuracy")

In [26]:
import os
import gzip
import json
import evaluate
import re
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
os.environ["WANDB_DISABLED"] = "true"


# === Step 2: Load and Prepare Data ===
url = "http://dl.turkunlp.org/tda-course-2025/tda25-responses.jsonl.gz"
filename = "tda25-responses.jsonl.gz"
os.system(f"wget {url} -O {filename}")

with gzip.open(filename, 'rt', encoding='utf-8') as f_in:
    data = [json.loads(line) for line in f_in]

df = pd.DataFrame(data)

# Function to extract 'Training' value using regex
def extract_training(response):
    match = re.search(r"Training:\s*(Yes|No)", response)
    return match.group(1) if match else None

# Apply the extraction function
df['Training'] = df['response'].apply(extract_training)

# Drop rows where Training is None (incomplete data)
df = df.dropna(subset=['Training'])

# Convert 'Training' to binary labels
df['label'] = df['Training'].map({'Yes': 1, 'No': 0})

# Final dataset with 'document' and 'label'
df = df[['document', 'label']]
print(df.head())
dataset = Dataset.from_pandas(df[['document', 'label']])

# Split the dataset
train_test = dataset.train_test_split(test_size=0.2)
train_dataset = train_test['train']
test_dataset = train_test['test']


# === Step 3: Train an Encoder-based Classifier ===
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['document'], truncation=True, padding="max_length", max_length=512)



train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

print(dataset.column_names)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to=[],  # Disable wandb reporting
    logging_steps=10,
)

# Load metrics using the 'evaluate' library
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average='binary')['precision'],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average='binary')['recall'],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average='binary')['f1'],
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)



                                            document  label
0  Peeling an onion seems like an trivial task, b...      0
1  Cowboy’s WR Terrance Williams Arrested\nDallas...      0
2  Idea Crib’s Well-thy Pinoy Profiles blog serie...      0
3  Will I ever stop being nervous every time I pu...      0
4  In today’s NHL rumors rundown, Toronto Maple L...      0


Map:   0%|          | 0/1994 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

['document', 'label', '__index_level_0__']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.228900,0.237690,0.877756,0.520408,0.784615,0.625767
2,0.246900,0.238907,0.885772,0.564516,0.538462,0.551181
3,0.183900,0.233551,0.885772,0.555556,0.615385,0.583942


TrainOutput(global_step=375, training_loss=0.23118883196512857, metrics={'train_runtime': 589.9039, 'train_samples_per_second': 10.141, 'train_steps_per_second': 0.636, 'total_flos': 1573930333163520.0, 'train_loss': 0.23118883196512857, 'epoch': 3.0})

In [28]:
# === Step 4: Evaluate and Analyze ===
eval_results = trainer.evaluate()
print("Evaluation Metrics:", eval_results)

Evaluation Metrics: {'eval_loss': 0.233550563454628, 'eval_accuracy': 0.8857715430861723, 'eval_precision': 0.5555555555555556, 'eval_recall': 0.6153846153846154, 'eval_f1': 0.583941605839416, 'eval_runtime': 13.3252, 'eval_samples_per_second': 37.448, 'eval_steps_per_second': 2.401, 'epoch': 3.0}


In [31]:




# Manual inspection of some predictions
# Perform predictions
predictions = trainer.predict(test_dataset)

# Extract predicted labels
preds = np.argmax(predictions.predictions, axis=-1)
# Create a comparison DataFrame
comparison = pd.DataFrame({
    "document": test_df['document'].values[:25],  # Use 'document' instead of 'article'
    "actual": test_df['label'].values[:25],
    "predicted": preds[:25]
})
print(comparison)

# Analyze data distribution
distribution = df['label'].value_counts(normalize=True)
print("Data Distribution (0=No, 1=Yes):", distribution)


                                             document  actual  predicted
0   Coming into New Castle, Kentucky on a brisk ni...       0          0
1   When is the Best Time to Start Performance Tes...       0          0
2   August 13, 2006\nHow to create a Linux screenc...       0          0
3   “It was nice to see Drew go out and run well a...       0          0
4   It was an hectic day yesterday, but filled wit...       0          0
5   If you have our "multiple Cohorts" feature ena...       1          0
6   1 and 1/2 Cups White Chocolate Chips\n2 Tables...       0          0
7   Okay, I admit, I eat WAY more ice cream than t...       0          0
8   Our Three Stooges\nIn honor of the new Three S...       0          0
9   Cocktail Series – Trunk helps you make it wet ...       1          0
10  Pictured Rocks Boat Cruise - Pictured Rocks Na...       0          0
11  Do you dream of leading a different kind of li...       0          0
12  The Super Eagles have been handed a welcome bo.